### Let's import the EarthEngine and geemap libraries

In [1]:
import ee
import geemap

### Create an interactive map

In [13]:
Map = geemap.Map(center=[40, -100], zoom=4)
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [14]:
# Import the NLCD collection.
dataset = ee.ImageCollection('USGS/NLCD_RELEASES/2019_REL/NLCD')

# Filter the collection to the 2019 product.
nlcd2019 = dataset.filter(ee.Filter.eq('system:index', '2019')).first()

# Select the land cover band.
landcover = nlcd2019.select('landcover')

# Display land cover on the map.
Map.addLayer(landcover, {}, 'NLCD 2019')

#Add built-in legend
title = 'NLCD Land Cover Classification'
#Map.add_legend(legend_title=title, builtin_legend='NLCD') 
#Map 

### Add custom legend

In [15]:
legend_dict = {
'11 Open Water': '466b9f',
'12 Perennial Ice/Snow': 'd1def8',
'21 Developed, Open Space': 'dec5c5',
'22 Developed, Low Intensity': 'd99282',
'23 Developed, Medium Intensity': 'eb0000',
'24 Developed High Intensity': 'ab0000',
'31 Barren Land (Rock/Sand/Clay)': 'b3ac9f',
'41 Deciduous Forest': '68ab5f',
'42 Evergreen Forest': '1c5f2c',
'43 Mixed Forest': 'b5c58f',
'51 Dwarf Scrub': 'af963c',
'52 Shrub/Scrub': 'ccb879',
'71 Grassland/Herbaceous': 'dfdfc2',
'72 Sedge/Herbaceous': 'd1d182',
'73 Lichens': 'a3cc51',
'74 Moss': '82ba9e',
'81 Pasture/Hay': 'dcd939',
'82 Cultivated Crops': 'ab6c28',
'90 Woody Wetlands': 'b8d9eb',
'95 Emergent Herbaceous Wetlands': '6c9fb8'
} 
title = 'NLCD Land Cover Classification'
Map.add_legend(legend_title=title, legend_dict=legend_dict)
Map

Map(bottom=1851.0, center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=S…

#### The map above shows only the NLCD 2019 image. To create an EE App for visualizing landcover change, we need a stack of NLCD images


In [17]:
# Let's print the list of systemIDs for all available NLCD images
dataset.aggregate_array('system:id').getInfo()

['USGS/NLCD_RELEASES/2019_REL/NLCD/2001',
 'USGS/NLCD_RELEASES/2019_REL/NLCD/2004',
 'USGS/NLCD_RELEASES/2019_REL/NLCD/2006',
 'USGS/NLCD_RELEASES/2019_REL/NLCD/2008',
 'USGS/NLCD_RELEASES/2019_REL/NLCD/2011',
 'USGS/NLCD_RELEASES/2019_REL/NLCD/2013',
 'USGS/NLCD_RELEASES/2019_REL/NLCD/2016',
 'USGS/NLCD_RELEASES/2019_REL/NLCD/2019']

In [19]:
#select the NLCD epochs after 2000
years = ['2001', '2004', '2006', '2008', '2011', '2013', '2016', '2019'] 

In [18]:
# Define the function for filtering the NLCD ImageCollection by year and select the landcover band
# Get an NLCD image by year.
def getNLCD(year):
    # Import the NLCD collection.
    dataset = ee.ImageCollection('USGS/NLCD_RELEASES/2019_REL/NLCD')
    # Filter the collection by year.
    nlcd = dataset.filter(ee.Filter.eq('system:index', year)).first()
    # Select the land cover band.
    landcover = nlcd.select('landcover');
    return landcover

In [20]:
# Create an NLCD image collection for the selected years.
collection = ee.ImageCollection(ee.List(years).map(lambda year: getNLCD(year)))

In [22]:
# Print out the list of system IDs of the selected NLCD images covering the contiguous United States.
collection.aggregate_array('system:id').getInfo() 

['USGS/NLCD_RELEASES/2019_REL/NLCD/2001',
 'USGS/NLCD_RELEASES/2019_REL/NLCD/2004',
 'USGS/NLCD_RELEASES/2019_REL/NLCD/2006',
 'USGS/NLCD_RELEASES/2019_REL/NLCD/2008',
 'USGS/NLCD_RELEASES/2019_REL/NLCD/2011',
 'USGS/NLCD_RELEASES/2019_REL/NLCD/2013',
 'USGS/NLCD_RELEASES/2019_REL/NLCD/2016',
 'USGS/NLCD_RELEASES/2019_REL/NLCD/2019']

In [23]:
# Create a list of labels to populate the drop down list
labels = [f'NLCD {year}' for year in years]
labels

['NLCD 2001',
 'NLCD 2004',
 'NLCD 2006',
 'NLCD 2008',
 'NLCD 2011',
 'NLCD 2013',
 'NLCD 2016',
 'NLCD 2019']

In [24]:
# The last step is to create a split-panel map by passing the NLCD ImageCollection and list of labels to ts_inspector
Map.ts_inspector(left_ts=collection, right_ts=collection, left_names=labels, right_names=labels)
Map 

Map(bottom=1851.0, center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=D…